In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/hate_speech_dataset.csv", index_col=0)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24783 entries, 0 to 25296
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   24783 non-null  int64 
 1   tweet   24783 non-null  object
dtypes: int64(1), object(1)
memory usage: 580.9+ KB


In [ ]:
for i in data['tweet'].head(10).tolist():
    print(i)
    print('------------------------------------------------------')

!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
------------------------------------------------------
!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
------------------------------------------------------
!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
------------------------------------------------------
!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
------------------------------------------------------
!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
------------------------------------------------------
!!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &#128514;&#128514;&#128514;"
-----------------

In [ ]:
# 0: hate speech, 1: offensive language, 2: neither
data['class'].value_counts()

1    19190
2     4163
0     1430
Name: class, dtype: int64

In [ ]:
#Preprocessing
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def preprocess_text(tweet):
    tweet = tweet.lower() #convert to lowercase
    tweet = re.sub(r'^rt[\s]+', '', tweet) #remove retweet
    tweet = re.sub(' rt ', '', tweet) #remove retweet

    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet) #remove hyperlink
    tweet = re.sub(r'#', '', tweet) #remove hashtag
    tweet = re.sub(r'[0-9]+', '', tweet) #remove numbers
    tweet = re.sub(r'[^\w\s]', '', tweet) #remove punctuation

    # return tweet

    stop_words = set(stopwords.words('english'))
    words = [word for word in tweet.split() if word not in stop_words]

    return ' '.join(words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data['cleaned_tweet'] = data['tweet'].apply(preprocess_text)

In [ ]:
for i in data['cleaned_tweet'].head(10).tolist():
    print(i)
    print('---------------------------------------------')

mayasolovely woman shouldnt complain cleaning house amp man always take trash
---------------------------------------------
mleew boy dats coldtyga dwn bad cuffin dat hoe st place
---------------------------------------------
urkindofbrand dawgsbabylife ever fuck bitch start cry confused shit
---------------------------------------------
c_g_anderson viva_based look like tranny
---------------------------------------------
shenikaroberts shit hear might true might faker bitch told ya
---------------------------------------------
t_madison_x shit blows meclaim faithful somebody still fucking hoes
---------------------------------------------
__brighterdays sit hate another bitch got much shit going
---------------------------------------------
selfiequeenbri cause im tired big bitches coming us skinny girls
---------------------------------------------
amp might get ya bitch back amp thats
---------------------------------------------
rhythmixx_ hobbies include fighting mariam bitch
---

In [ ]:
data = data.sample(5000, random_state = 40)
# data = data[:5000].reset_index()
data = data.reset_index()
data.head()

,index,class,tweet,cleaned_tweet
0,21823,1,That bitch called on Thor for some strength. &...,bitch called thor strength rtchubbygirlgod wait
1,11160,1,"I shoot at the pussy, I bust in the pussy I'm ...",shoot pussy bust pussy im cuming sooooooooooon
2,19924,1,RT @lamessican: I love when bitches throw shad...,lamessican love bitches throw shade confirms i...
3,20360,2,RT @rickygervais: Humans are genetically close...,rickygervais humans genetically closer chimps ...
4,13633,1,Niggahs need to keep there bitches in check ',niggahs need keep bitches check


In [ ]:
data['class'].value_counts()

1    3857
2     850
0     293
Name: class, dtype: int64

In [ ]:
for i, j in zip(data['cleaned_tweet'].head(10).tolist(), data['tweet'].head(10).tolist()):
    print(i)
    print('------------------------------------------------------')
    print(j)
    print('------------------------------------------------------')
    print('------------------------------------------------------')

bitch called thor strength rtchubbygirlgod wait
------------------------------------------------------
That bitch called on Thor for some strength. &#58386;&#58386; RT"@ChubbyGirlGod: Wait... http://t.co/UAf57JyiOi"
------------------------------------------------------
------------------------------------------------------
shoot pussy bust pussy im cuming sooooooooooon
------------------------------------------------------
I shoot at the pussy, I bust in the pussy I'm cuming too sooooooooooon
------------------------------------------------------
------------------------------------------------------
lamessican love bitches throw shade confirms im something right
------------------------------------------------------
RT @lamessican: I love when bitches throw shade. Just confirms i'm doing something right &#128514;
------------------------------------------------------
------------------------------------------------------
rickygervais humans genetically closer chimps chimps gorillas a

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['cleaned_tweet'], data['class'], test_size=0.2, random_state=40)

X_val = X_train[:int(len(X_train)*(1/8))]
y_val = y_train[:int(len(y_train)*(1/8))]
X_train = X_train[int(len(X_train)*(1/8)):]
y_train = y_train[int(len(y_train)*(1/8)):]

print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_val shape: ", X_val.shape)
print("y_val shape: ", y_val.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)


X_train shape:  (3500,)
y_train shape:  (3500,)
X_val shape:  (500,)
y_val shape:  (500,)
X_test shape:  (1000,)
y_test shape:  (1000,)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import transformers
from transformers import BertTokenizer

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def tokenize_texts(texts):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_text = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=32,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        input_ids.append(encoded_text['input_ids'])
        attention_masks.append(encoded_text['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_input_ids, train_attention_masks = tokenize_texts(X_train)
val_input_ids, val_attention_masks = tokenize_texts(X_val)
test_input_ids, test_attention_masks = tokenize_texts(X_test)

train_labels = torch.tensor(y_train.values)
val_labels = torch.tensor(y_val.values)
test_labels = torch.tensor(y_test.values)

train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
class TextClassifierModel(nn.Module):

    def __init__(self):
        super(TextClassifierModel, self).__init__()

        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased')

        # freeze bert layers
        # for param in self.bert.parameters():
        #     param.requires_grad = False

        self.cnn = nn.Conv1d(768, 256, kernel_size=3, padding=1)
        self.bigru = nn.GRU(input_size=768, hidden_size=768, num_layers=2, bidirectional=True)

        # Calculate the input size for the fully connected layer
        fc_input_size = (256 + 2 * 768)*32

        self.fc = nn.Linear(fc_input_size, 3)
        # self.activation = nn.ReLU()  # You can use another activation function if needed

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_output = bert_output.last_hidden_state.transpose(1, 2)

        cnn_output = self.cnn(bert_output)
        cnn_output = torch.flatten(cnn_output, start_dim=1)  # Flatten the CNN output

        gru_output, _ = self.bigru(bert_output.permute(0, 2, 1))
        gru_output = gru_output.permute(0, 2, 1)
        gru_output = torch.flatten(gru_output, start_dim=1)  # Flatten the GRU output

        cat_output = torch.cat((cnn_output, gru_output), dim=1)

        final_output = self.fc(cat_output)
        # final_output = self.activation(final_output)  # Apply activation function

        return final_output

In [ ]:
from sklearn.metrics import classification_report

model = TextClassifierModel()
model = model.to(device)

optim = torch.optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(5):
    model.train()
    i = 0
    for batch in train_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        output = model(input_ids, attention_mask)

        loss = loss_fn(output, labels)
        loss.backward()
        optim.step()
        optim.zero_grad()
        print(i, " Training Loss: ", loss)
        i+=1

    model.eval()
    with torch.no_grad():
        total_preds, total_labels = [], []
        for batch in val_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)
            total_labels.append(labels.cpu().numpy())

            output = model(input_ids, attention_mask)
            loss = loss_fn(output, labels)
            # print("Validation Loss: ", loss)

            preds = torch.argmax(output, dim=1)
            total_preds.append(preds.cpu().numpy())

        total_labels, total_preds = np.concatenate(total_labels), np.concatenate(total_preds)
        print(classification_report(total_labels, total_preds))

0  Training Loss:  tensor(1.0425, device='cuda:0', grad_fn=<NllLossBackward0>)
1  Training Loss:  tensor(0.7755, device='cuda:0', grad_fn=<NllLossBackward0>)
2  Training Loss:  tensor(0.7145, device='cuda:0', grad_fn=<NllLossBackward0>)
3  Training Loss:  tensor(0.3828, device='cuda:0', grad_fn=<NllLossBackward0>)
4  Training Loss:  tensor(0.8021, device='cuda:0', grad_fn=<NllLossBackward0>)
5  Training Loss:  tensor(1.5612, device='cuda:0', grad_fn=<NllLossBackward0>)
6  Training Loss:  tensor(0.8675, device='cuda:0', grad_fn=<NllLossBackward0>)
7  Training Loss:  tensor(0.8361, device='cuda:0', grad_fn=<NllLossBackward0>)
8  Training Loss:  tensor(0.1883, device='cuda:0', grad_fn=<NllLossBackward0>)
9  Training Loss:  tensor(0.1523, device='cuda:0', grad_fn=<NllLossBackward0>)
10  Training Loss:  tensor(0.4611, device='cuda:0', grad_fn=<NllLossBackward0>)
11  Training Loss:  tensor(0.4215, device='cuda:0', grad_fn=<NllLossBackward0>)
12  Training Loss:  tensor(1.4138, device='cuda:0'

In [ ]:
total_preds, total_labels = [], []
for batch in test_dataloader:
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to(device)
    total_labels.append(labels.cpu().numpy())

    output = model(input_ids, attention_mask)
    loss = loss_fn(output, labels)
    # print("Validation Loss: ", loss)

    preds = torch.argmax(output, dim=1)
    total_preds.append(preds.cpu().numpy())

total_labels, total_preds = np.concatenate(total_labels), np.concatenate(total_preds)
print(classification_report(total_labels, total_preds))

              precision    recall  f1-score   support

           0       0.46      0.34      0.39        53
           1       0.94      0.95      0.94       778
           2       0.85      0.88      0.86       169

    accuracy                           0.90      1000
   macro avg       0.75      0.72      0.73      1000
weighted avg       0.90      0.90      0.90      1000



In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(total_labels, total_preds)

0.902

In [ ]:
zer, one, two = 0, 0, 0
for i in total_labels:
    if i == 0:
        zer+=1
    elif i == 1:
        one+=1
    else:
        two+=1
print(zer, one, two)

53 778 169


In [ ]:
zer, one, two = 0, 0, 0
for i in total_preds:
    if i == 0:
        zer+=1
    elif i == 1:
        one+=1
    else:
        two+=1
print(zer, one, two)

39 787 174
